In [21]:
## init envi if local
#!pip install torch==1.5.1+cu92 torchvision==0.6.1+cu92 -f https://download.pytorch.org/whl/torch_stable.html


In [22]:
## use colab
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [23]:
!ls gdrive/MyDrive/DEEPCROP_Lei/reg/reg_pytorch_lidar
os.chdir('/content/gdrive/MyDrive/DEEPCROP_Lei/reg/reg_pytorch_lidar/')
!pwd

ls: cannot access 'gdrive/MyDrive/DEEPCROP_Lei/reg/reg_pytorch_lidar': No such file or directory
/content/gdrive/MyDrive/DEEPCROP_Lei/reg/reg_pytorch_lidar


###  step by step

In [24]:
# dataloader for regression
import os
import numpy as np
from torch.utils.data import Dataset
import pdb
import random
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

#Standardize the entire data set before dividing it

# Standardize the training level before applying the rules to the test set (TODO)

class GeoData_crop_1(Dataset):
    def __init__(self, split='train', data_root='trainval_fullarea', num_point=4096,  block_size=1.0,  transform=None):
        super().__init__()
        self.num_point = num_point
        self.block_size = block_size
        self.transform = transform
        rooms = sorted(os.listdir(data_root))
        rooms_split = sorted(os.listdir(data_root))

        self.room_points, self.room_labels = [], []
        self.room_coord_min, self.room_coord_max = [], []
        num_point_all = []
        self.room_points_1, self.room_labels_1 = [], []
        self.intensity_all = []
        # for room_name in rooms_split:
        #     room_path = os.path.join(data_root, room_name)
        #     # room_data = np.load(room_path)  # xyzrgbl, N*7
        #     room_data = pd.read_csv(room_path, sep="\s+").values  # xyzrgbl, N*7
        #     # print(room_path)

        #     points, labels = room_data[:, 0:3], room_data[:, -1] 
        #     rgb = room_data[:,10:13]
        #     intensity = room_data[:,3].reshape(points.shape[0], 1)
        #     # self.intensity_all.append(intensity.tolist())
        #     coord_min, coord_max = np.amin(points, axis=0)[:3], np.amax(points, axis=0)[:3]
        #     rgb = rgb / (256 * 255)
        #     self.room_coord_min.append(coord_min), self.room_coord_max.append(coord_max)
        #     points = np.concatenate((points, rgb, intensity), axis=1)

        #     self.room_points_1.append(points)
        #     for i in labels.tolist():
        #         self.room_labels_1.append(i)
        #     for j in intensity.tolist():
        #         self.intensity_all.append(j)
                
        # whole_coord_min, whole_coord_max = np.amin(self.room_coord_min, axis=0)[:3], np.amax(self.room_coord_max, axis=0)[:3]
        # room_scale = (whole_coord_max - whole_coord_min)
        # # normalize intensity
        # intensity_mean = np.array(self.intensity_all).mean()
        # intensity_std = np.array(self.intensity_all).std()

        # # normalize label
        # room_labels_mean = np.array(self.room_labels_1).mean()
        # room_labels_std = np.array(self.room_labels_1).std()

        ### whole data 
        intensity_mean = 30.015351566593313
        intensity_std = 2.2168713538563276

        # normalize label
        room_labels_mean = 925.8869876854762
        room_labels_std =  499.07046037234073
        for room_name in rooms_split:
            room_path = os.path.join(data_root, room_name)
            room_data = pd.read_csv(room_path, sep="\s+").values  # xyzrgbl, N*7

            points, labels = room_data[:, 0:3], room_data[:, -1]
            rgb = room_data[:, 10:13]
            intensity = room_data[:, 3].reshape(points.shape[0], 1)

            intensity = (intensity - intensity_mean) / intensity_std
            labels = (labels - room_labels_mean) / room_labels_std

            # normalize
            points = (points - whole_coord_min) / room_scale
            rgb = rgb / (256 * 255)

            # self.room_coord_min.append(coord_min), self.room_coord_max.append(coord_max)
            num_point_all.append(labels.size)
#             points = np.concatenate((points, rgb, intensity), axis=1)
            self.room_points.append(points), self.room_labels.append(labels)        # normalize rgb

        room_idxs = []
        for index in range(len(rooms_split)):
            room_idxs.extend([index])
        self.room_idxs = np.array(room_idxs)
        print("Totally {} samples in {} set.".format(len(self.room_idxs), split))


    def __getitem__(self, idx):
        room_idx = self.room_idxs[idx]
        points = self.room_points[room_idx]   # N * 6
        labels = self.room_labels[room_idx]   # N
        N_points = points.shape[0]
        
        return points, labels

    def __len__(self):
        return len(self.room_idxs)

print("load dataloader methods")


load dataloader methods


In [25]:
# model init
## the basic network

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F


class STN3d(nn.Module):
    def __init__(self, channel):
        super(STN3d, self).__init__()
        # self.conv1 = torch.nn.Conv1d(channel, 64, 1)
        self.conv1 = torch.nn.Conv1d(channel, 64, 1)

        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 9)
        self.relu = nn.ReLU()

        # self.bn1 = nn.BatchNorm1d(64)
        # self.bn2 = nn.BatchNorm1d(128)
        # self.bn3 = nn.BatchNorm1d(1024)
        # self.bn4 = nn.BatchNorm1d(512)
        # self.bn5 = nn.BatchNorm1d(256)

    def forward(self, x):
        batchsize = x.size()[0]
        # x = F.relu(self.bn1(self.conv1(x)))
        # x = F.relu(self.bn2(self.conv2(x)))
        # x = F.relu(self.bn3(self.conv3(x)))
        # x = torch.max(x, 2, keepdim=True)[0]
        # x = x.view(-1, 1024)
        #
        # x = F.relu(self.bn4(self.fc1(x)))
        # x = F.relu(self.bn5(self.fc2(x)))
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.array([1, 0, 0, 0, 1, 0, 0, 0, 1]).astype(np.float32))).view(1, 9).repeat(
            batchsize, 1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, 3, 3)
        return x


class STNkd(nn.Module):
    def __init__(self, k=64):
        super(STNkd, self).__init__()
        self.conv1 = torch.nn.Conv1d(k, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k * k)
        self.relu = nn.ReLU()

        # self.bn1 = nn.BatchNorm1d(64)
        # self.bn2 = nn.BatchNorm1d(128)
        # self.bn3 = nn.BatchNorm1d(1024)
        # self.bn4 = nn.BatchNorm1d(512)
        # self.bn5 = nn.BatchNorm1d(256)

        self.k = k

    def forward(self, x):
        batchsize = x.size()[0]
        # x = F.relu(self.bn1(self.conv1(x)))
        # x = F.relu(self.bn2(self.conv2(x)))
        # x = F.relu(self.bn3(self.conv3(x)))
        # x = torch.max(x, 2, keepdim=True)[0]
        # x = x.view(-1, 1024)
        #
        # x = F.relu(self.bn4(self.fc1(x)))
        # x = F.relu(self.bn5(self.fc2(x)))
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.eye(self.k).flatten().astype(np.float32))).view(1, self.k * self.k).repeat(
            batchsize, 1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x


class PointNetEncoder(nn.Module):
    def __init__(self, global_feat=True, feature_transform=False, channel=3):
        super(PointNetEncoder, self).__init__()
        self.stn = STN3d(channel)
        self.conv1 = torch.nn.Conv1d(channel, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        # self.bn1 = nn.BatchNorm1d(64)
        # self.bn2 = nn.BatchNorm1d(128)
        # self.bn3 = nn.BatchNorm1d(1024)
        self.global_feat = global_feat
        self.feature_transform = feature_transform
        if self.feature_transform:
            self.fstn = STNkd(k=64)

    def forward(self, x):
        B, D, N = x.size()

        # print(x.size())
        trans = self.stn(x)

        x = x.transpose(2, 1)
        if D > 3:
            # x, feature = x.split(3, dim=2)
            x, feature = x.split((3,4), dim=2)

        x = torch.bmm(x, trans)
        if D > 3:
            x = torch.cat([x,feature], dim=2)
        x = x.transpose(2, 1)
        # x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.conv1(x))

        if self.feature_transform:
            trans_feat = self.fstn(x)
            x = x.transpose(2, 1)
            x = torch.bmm(x, trans_feat)
            x = x.transpose(2, 1)
        else:
            trans_feat = None

        pointfeat = x
        # x = F.relu(self.bn2(self.conv2(x)))
        # x = self.bn3(self.conv3(x))
        x = F.relu(self.conv2(x))
        x = self.conv3(x)
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        if self.global_feat:
            return x, trans, trans_feat
        else:
            x = x.view(-1, 1024, 1).repeat(1, 1, N)
            return torch.cat([x, pointfeat], 1), trans, trans_feat


def feature_transform_reguliarzer(trans):
    d = trans.size()[1]
    I = torch.eye(d)[None, :, :]
    if trans.is_cuda:
        I = I.cuda()
    loss = torch.mean(torch.norm(torch.bmm(trans, trans.transpose(2, 1) - I), dim=(1, 2)))
    return loss

print("model call")

model call


In [26]:
# the whole model network for regression task
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F

class get_model(nn.Module):
    def __init__(self, k=40, normal_channel=True):
        super(get_model, self).__init__()
        if normal_channel:
            channel = 7
        else:
            channel = 3
        channel = 3
        print(channel)

        self.feat = PointNetEncoder(global_feat=True, feature_transform=True, channel=channel)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k)
     
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x, trans, trans_feat = self.feat(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.dropout(self.fc2(x)))
        x = self.fc3(x)
        # pdb.set_trace()
        # x = F.log_softmax(x, dim=1)


        return x, trans_feat

class get_loss(torch.nn.Module):
    def __init__(self, mat_diff_loss_scale=0.001):
        super(get_loss, self).__init__()
        self.mat_diff_loss_scale = mat_diff_loss_scale

    def forward(self, pred, target, trans_feat):
        # loss = F.nll_loss(pred, target)
        # target = (2000 -target) / 2000.
#         pdb.set_trace()
        loss = F.mse_loss(pred, target)
#         loss = F.smooth_l1_loss(pred, target)


        mat_diff_loss = feature_transform_reguliarzer(trans_feat)

#         total_loss = loss + mat_diff_loss * self.mat_diff_loss_scale
        # pdb.set_trace()
        total_loss = loss
        return total_loss
print("load metric")

load metric


In [27]:
'''HYPER PARAMETER'''
import datetime
from pathlib import Path
import logging


def log_string(str):
    logger.info(str)
    print(str)
    
# os.environ["CUDA_VISIBLE_DEVICES"] = '0'
log_dir = 'rgn_point'
model = 'pointnet_reg_sml1BN1_point'
BATCH_SIZE = 1
epoch = 200
learning_rate = 0.0001
decay_rate = 0.7
step_size = 5
gpu = '0'
optimizer = 'Adam' # 'SGD'
lr_decay = 0.7
NUM_POINT, Block_size = 500, 2 # doesn't matter now


'''CREATE DIR'''
timestr = str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M'))
experiment_dir = Path('./log/')
experiment_dir.mkdir(exist_ok=True)
experiment_dir = experiment_dir.joinpath('reg')
experiment_dir.mkdir(exist_ok=True)
if log_dir is None:
    experiment_dir = experiment_dir.joinpath(timestr)
else:
    experiment_dir = experiment_dir.joinpath(log_dir)
experiment_dir.mkdir(exist_ok=True)
checkpoints_dir = experiment_dir.joinpath('checkpoints/')
checkpoints_dir.mkdir(exist_ok=True)
log_dir = experiment_dir.joinpath('logs/')
log_dir.mkdir(exist_ok=True)

'''LOG'''

logger = logging.getLogger("Model")
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler = logging.FileHandler('%s/%s.txt' % (log_dir, model))
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
log_string('PARAMETER ...')


PARAMETER ...


In [28]:
## load data
from torch.utils.tensorboard import SummaryWriter
import torch
import logging
import sys
import importlib
import shutil
from tqdm import tqdm
# import provider
import numpy as np
import time
%ls 
root = './data/train_test_split_v2/train_set_txt'
val_root = './data/train_test_split_v2/val_set_txt'
test_root = './data/train_test_split_v2/test_set_txt'

TRAIN_DATASET = GeoData_crop_1(split='train', data_root=root, num_point=NUM_POINT,  block_size=Block_size, transform=None)
print("start loading test data ...")
VAL_DATASET = GeoData_crop_1(split='test', data_root=val_root, num_point=NUM_POINT,  block_size=Block_size, transform=None)
TEST_DATASET = GeoData_crop_1(split='test', data_root=test_root, num_point=NUM_POINT,  block_size=Block_size, transform=None)

trainDataLoader = torch.utils.data.DataLoader(TRAIN_DATASET, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True, drop_last=True, worker_init_fn = lambda x: np.random.seed(x+int(time.time())))
valDataLoader = torch.utils.data.DataLoader(VAL_DATASET, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True, drop_last=True)
testDataLoader = torch.utils.data.DataLoader(TEST_DATASET, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True, drop_last=True)


AGB_reg_pointnet.ipynb  models/       test_reg_igndata_block_BN1_point.py
data/                   provider.py   test_reg_igndata_block_BN1_point_rgb.py
data_utils/             __pycache__/  train_reg_igndata_block_BN1_point.py
log/                    scripts/      train_reg_igndata_block_BN1_point_rgb.py


NameError: ignored

In [ ]:
# load model and training
classifier = get_model(1).cuda()
# load metric
criterion = get_loss().cuda()
from sklearn.metrics import r2_score

# weight init
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv2d') != -1:
        torch.nn.init.xavier_normal_(m.weight.data)
        torch.nn.init.constant_(m.bias.data, 0.0)
    elif classname.find('Linear') != -1:
        torch.nn.init.xavier_normal_(m.weight.data)
        torch.nn.init.constant_(m.bias.data, 0.0)

classifier = classifier.apply(weights_init)

# chose optimizer
if optimizer == 'Adam':
    optimizer = torch.optim.Adam(
        classifier.parameters(),
        lr=learning_rate,
        betas=(0.9, 0.999),
        eps=1e-08,
        weight_decay=decay_rate
    )
else:
    optimizer = torch.optim.SGD(classifier.parameters(), lr=learning_rate, momentum=0.9)

def bn_momentum_adjust(m, momentum):
    if isinstance(m, torch.nn.BatchNorm2d) or isinstance(m, torch.nn.BatchNorm1d):
        m.momentum = momentum
        
## define the training parameter

LEARNING_RATE_CLIP = 1e-7
MOMENTUM_ORIGINAL = 0.1
MOMENTUM_DECCAY = 0.5
MOMENTUM_DECCAY_STEP = step_size

print('load model and training')

In [ ]:
## start training
start_epoch = 0
global_epoch = 0
best_loss = 99999

TIMESTAMP = "{0:%Y-%m-%dT%H-%M-%S/}".format(datetime.datetime.now())
writer = SummaryWriter(str(experiment_dir) + "/tensorboard/" + TIMESTAMP)
for epoch in range(start_epoch, epoch):
    log_string('**** Epoch %d (%d/%s) ****' % (global_epoch + 1, epoch + 1, epoch))
    lr = max(learning_rate * (lr_decay ** (epoch // step_size)), LEARNING_RATE_CLIP)
    log_string('Learning rate:%f' % lr)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    momentum = MOMENTUM_ORIGINAL * (MOMENTUM_DECCAY ** (epoch // MOMENTUM_DECCAY_STEP))
    if momentum < 0.01:
        momentum = 0.01
    print('BN momentum updated to: %f' % momentum)
    classifier = classifier.apply(lambda x: bn_momentum_adjust(x,momentum))
    num_batches = len(trainDataLoader)
    total_correct = 0
    total_seen = 0
    loss_sum = 0
    
    ### load batches
    for i, data in tqdm(enumerate(trainDataLoader), total=len(trainDataLoader), smoothing=0.9):
        points, target = data
        points = points.data.numpy()
        # points[:,:, :3] = provider.rotate_point_cloud_z(points[:,:, :3])

        target = target.data.numpy()
        target = torch.Tensor(target)

        points = torch.Tensor(points)
        points, target = points.float().cuda(),target.float().cuda()
        points = points.transpose(2, 1)
        optimizer.zero_grad()

        classifier = classifier.train()
        reg_pred, trans_feat = classifier(points)
        reg_pred = reg_pred.contiguous().view(-1)

        # batch_label = target.view(-1, 1)[:, 0].cpu().data.numpy()
        # target = target.view(-1, 1)[:, 0]
        target = target[:, 0]
        # pdb.set_trace()

        loss = criterion(reg_pred, target.float(), trans_feat)
        loss.backward()
        optimizer.step()
        loss_sum += loss
    avl_loss = loss_sum/num_batches

    writer.add_scalar('Train_Loss', avl_loss, epoch)
    writer.add_scalar('learning_rate',lr , epoch)


    log_string('==========Training mean loss: %f ================' % (avl_loss))
    
    ## save models
    if epoch % 50 == 0:
        logger.info('Save model...')
        # pdb.set_trace()
        savepath = str(checkpoints_dir) + '/' + str(epoch) + '_model.pth'
        log_string('Saving at %s' % savepath)
        state = {
            'epoch': epoch,
            'model_state_dict': classifier.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }
        torch.save(state, savepath)
        log_string('Saving model....')

        
    '''Evaluate on validate data'''
    with torch.no_grad():
        num_batches = len(valDataLoader)
        # total_correct = 0
        # total_seen = 0
        loss_sum = 0
        log_string('---- EPOCH %03d EVALUATION ----' % (global_epoch + 1))
        for i, data in tqdm(enumerate(valDataLoader), total=len(valDataLoader), smoothing=0.9):
            points, target = data
            points = points.data.numpy()
            points = torch.Tensor(points)
            # target to tensor
            target = target.data.numpy()
            target = torch.Tensor(target)
            points, target = points.float().cuda(), target.float().cuda()
            points = points.transpose(2, 1)
            classifier = classifier.eval()
            reg_pred, trans_feat = classifier(points)
            reg_pred = reg_pred.contiguous().view(-1)
            batch_label = target.cpu().data.numpy()

            target = target[:, 0]

            loss = criterion(reg_pred, target.float(), trans_feat)
            loss_sum += loss
        avl_loss = loss_sum/num_batches
        log_string('=================== Test mean loss: %f ==============' % (avl_loss))

        writer.add_scalar('Val_Loss', avl_loss, epoch)

        # log_string('eval mean loss: %f' % (loss_sum / float(num_batches)))
        if avl_loss <= best_loss:
            best_loss = avl_loss
            best_epoch = epoch
            logger.info('Save model...')
            savepath = str(checkpoints_dir) + '/best_model.pth'
            log_string('Saving at %s' % savepath)
            state = {
                'epoch': epoch,
                'class_avg_iou': loss_sum,
                'model_state_dict': classifier.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }
            torch.save(state, savepath)
            log_string('Saving model....')
        log_string('Best loss: %f' % best_loss)
        log_string('Best loss epoch: %f' % best_epoch)

    ## new add
    '''Evaluate on test data and compute the r2-scores'''
    r2_score_sum = 0

    with torch.no_grad():
        num_batches = len(testDataLoader)
        # total_correct = 0
        # total_seen = 0
        loss_sum = 0
        log_string('---- EPOCH %03d test data ----' % (global_epoch + 1))
        target_all = list()
        reg_all = list()
        for i, data in tqdm(enumerate(testDataLoader), total=len(testDataLoader), smoothing=0.9):
            points, target = data
            points = points.data.numpy()
            points = torch.Tensor(points)
            # target to tensor
            target = target.data.numpy()
            target = torch.Tensor(target)
            points, target = points.float().cuda(), target.float().cuda()
            points = points.transpose(2, 1)
            classifier = classifier.eval()
            reg_pred, trans_feat = classifier(points)
            reg_pred = reg_pred.contiguous().view(-1)
            batch_label = target.cpu().data.numpy()
            target = target[:, 0].float()
            loss = criterion(reg_pred, target.float(), trans_feat)
            loss_sum += loss

            target_all.append(target.cpu())
            reg_all.append(reg_pred.cpu())

        r2_score_all = r2_score(target_all, reg_all)

        avl_loss = loss_sum/num_batches
        log_string('=================== Test mean loss: %f ==============' % (avl_loss))
        log_string('=================== Test r2_score_all: %f ==============' % (r2_score_all))

        writer.add_scalar('Test_Loss', avl_loss, epoch)
        writer.add_scalar('Test_r2_scores',r2_score_all , epoch)

    global_epoch += 1
writer.close()

In [ ]:
# view the training process
#!tensorboard --logdir=log/reg/rgn_point/tensorboard

In [ ]:
## testing the trained model
import os
import torch
import datetime
import logging
from pathlib import Path
import sys
from tqdm import tqdm
import provider
import numpy as np
import time
from sklearn.metrics import r2_score


test_root = '/data/REASEARCH/DEEPCROP/PointCloudData/regression/train_test_split_v2/test_set_txt'
TEST_DATASET = GeoData_crop_1(split='test', data_root=test_root, num_point=NUM_POINT,  block_size=Block_size, transform=None)
testDataLoader = torch.utils.data.DataLoader(TEST_DATASET, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True, drop_last=True)

# load model and training
classifier = get_model(1).cuda()
# load metric
criterion = get_loss().cuda()
checkpoint = torch.load(str(experiment_dir) + '/checkpoints/best_model.pth')

start_epoch = checkpoint['epoch']
classifier.load_state_dict(checkpoint['model_state_dict'])

for i, data in tqdm(enumerate(testDataLoader), total=len(testDataLoader), smoothing=0.9):
    points, target = data
    points = points.data.numpy()
    points = torch.Tensor(points)
    # target to tensor
    target = target.data.numpy()
    target = torch.Tensor(target)
    points, target = points.float().cuda(), target.float().cuda()
    points = points.transpose(2, 1)
    classifier = classifier.eval()
    seg_pred, trans_feat = classifier(points)
    # pred_val = seg_pred.contiguous().cpu().data.numpy()
    seg_pred = seg_pred.contiguous().view(-1)
    batch_label = target.cpu().data.numpy()
    # target = target.view(-1, 1)[:, 0]

    target = target[:, 0]

    loss = criterion(seg_pred, target.float(), trans_feat)
    loss_sum += loss

    target_all.append(target.cpu())
    reg_all.append(seg_pred.cpu())

r2_score_all = r2_score(target_all, reg_all)

# avl_r2_score = r2_score_sum/num_batches
avl_loss = loss_sum/num_batches
log_string('===================  Test mean L2 loss: %f  ==============' % (avl_loss))
log_string('===================  Test mean r2 score: %f  ==============' % (r2_score_all))
log_string('===================  Test mean L1 loss: %f  ==============' % (avl_loss))

In [ ]:
##########  use terminal

In [ ]:
!sh scripts/Train_ign_point.sh

In [ ]:
!sh scripts_0526/Test_point.sh

In [ ]:
#!tensorboard --logdir=log/reg/rgn_point/tensorboard